# Find Local Max and Separate Foci, features extraction

## Find Local Max and Separate Foci

In [ ]:
from skimage.feature import peak_local_max
from skimage.io import imread, imshow
from skimage.filters import gaussian 
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage.segmentation import watershed
from skimage.measure import regionprops
from skimage.measure import regionprops_table
import ipywidgets as widgets
from IPython.display import display
import tifffile
import numpy as np
import pandas as pd
import glob
import os
from datetime import datetime

In [ ]:
today_date = datetime.now().strftime("%d_%m_%Y") # data

#output_folder_name = today_date + "_cells"

current_directory = os.getcwd()
#output_folder_path = os.path.join(current_directory, "output\\", output_folder_name) 
output_folder_path = os.path.join(current_directory, "labels")
input_folder_path = os.path.join(current_directory, "BDS3_2025_data", "MXT")
masks_folder_path = os.path.join(current_directory, "masks")
hist_folder_path = os.path.join(current_directory, "data", "histograms")


folders_to_check = [output_folder_path, input_folder_path,masks_folder_path, hist_folder_path]

for folder in folders_to_check:
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(f"Created folder: {folder}")
    else:
        print(f"Folder already exists: {folder}")

Folder already exists: /Users/khristinafediv/Downloads/Python/image_analysis_ubds/labels
Folder already exists: /Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT
Folder already exists: /Users/khristinafediv/Downloads/Python/image_analysis_ubds/masks
Folder already exists: /Users/khristinafediv/Downloads/Python/image_analysis_ubds/data/histograms


In [ ]:
# Lists for storing processed DataFrames
tif_files = [] 

#Iteration by files in a folder
for file_name in os.listdir(input_folder_path): 
    if file_name.lower().endswith(('.tif', '.tiff')): 
        file_path = os.path.join(input_folder_path, file_name)
        tif_files.append(file_path)  #add full path to a file
        
print("Files found:")
for f in tif_files:
    print(f)

Files found:
/Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT/2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C1.tif
/Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT/2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C3.tif
/Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT/2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C2.tif
/Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT/2015.01.17_MTX_1h-1h_S_I_Pos001_S001_0_C1.tif
/Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT/2015.01.17_MTX_1h-1h_S_I_Pos001_S001_0_C2.tif
/Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT/2015.01.17_MTX_1h-1h_S_I_Pos001_S001_0_C3.tif
/Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT/2015.01.17_MTX_1h-1h_S_IV_Pos001_S001_9_C1.tif
/Users/khristinafediv/Downloads/Python/image_analysis_ubds/BDS3_2025_data/MXT/2015.01.18_MTX_1h-1h_S_III_Pos030_S001_0

In [5]:
for file in tif_files: 
    image = imread(file)
    img_name = os.path.basename(file)
    base_name, ext = os.path.splitext(img_name)
    print(f"Image name: {img_name}, shape: {image.shape}")

    mask_path = os.path.join(masks_folder_path, img_name)
    mask_name = base_name + '.tiff'
    mask_path2 = os.path.join(masks_folder_path, mask_name)
    
    if os.path.exists(mask_path):
        binary_mask = imread(mask_path)
        print(f"Image name: {img_name}, shape: {image.shape}. Mask for the image found ({binary_mask.shape})")
    elif os.path.exists(mask_path2):
        binary_mask = imread(mask_path2)
        print(f"Image name: {img_name}, shape: {image.shape}. Mask found: {mask_path2} ({binary_mask.shape})")
    else:
        print(f"Missing mask for the image {img_name}!")
        break
        
    #Gaussian blur - image preprocessing 
    preprocessed = gaussian(image, sigma=1.5, preserve_range=True)
    print(f"min: {preprocessed.min()}, max: {preprocessed.max()}")
    
    #Create histograms of intensities
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    axs[0].hist(preprocessed.ravel(), bins=100)
    axs[0].set_title("Histogram (linear scale)")
    axs[0].set_xlabel("Intensity")
    axs[0].set_ylabel("Count")

    axs[1].hist(preprocessed.ravel(), bins=100)
    axs[1].set_title("Histogram (log scale)")
    axs[1].set_xlabel("Intensity")
    axs[1].set_yscale('log')
    axs[1].set_xlim(left=0)
    plt.tight_layout()
    #save as png
    save_path = os.path.join(hist_folder_path, img_name.replace('.tif', '_hist.png').replace('.tiff', '_hist.png'))
    plt.savefig(save_path)
    plt.close()
    print(f"Histogram saved to: {hist_folder_path}")

    coordinates = peak_local_max(preprocessed, threshold_abs=4, footprint=np.ones((4, 4, 4)))
    print(f"Nr of max found: {len(coordinates)}")

    #image = 3D array with intensities
    #binary_mask = 3D binary mask (1 - object, 0 - background)
    #coordinates = list of coordinates of local maxima (e.g., [(z1,y1,x1), (z2,y2,x2), ...])

    #create labels (label image) from seed points
    markers = np.zeros_like(binary_mask, dtype=np.int32)
    for i, (z, y, x) in enumerate(coordinates, 1):  #enumerate starting from 1
        markers[z, y, x] = i

    #set the mask where the growth is allowed
    mask = binary_mask.astype(bool)

    #watershed, with the usage of the original image 
    labels = watershed(-image, markers=markers, mask=mask)

    #to save labels - folder "./labels" should be created
    labels_path = os.path.join(output_folder_path, img_name.replace('.tif', '_labels.tif').replace('.tiff', '_labels.tif'))
    tifffile.imwrite(labels_path, labels.astype(np.uint16))

    print(f"Labels saved for the image {img_name}.")

print(f"All done.")

Image name: 2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C1.tif, shape: (82, 512, 512)
Image name: 2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C1.tif, shape: (82, 512, 512). Mask for the image found ((82, 512, 512))
min: 0.0, max: 49.28575871799903
Histogram saved to: /Users/khristinafediv/Downloads/Python/image_analysis_ubds/data/histograms
Nr of max found: 263
Labels saved for the image 2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C1.tif.
Image name: 2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C3.tif, shape: (82, 512, 512)
Image name: 2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C3.tif, shape: (82, 512, 512). Mask for the image found ((82, 512, 512))
min: 0.0, max: 124.92432881657481
Histogram saved to: /Users/khristinafediv/Downloads/Python/image_analysis_ubds/data/histograms
Nr of max found: 808
Labels saved for the image 2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C3.tif.
Image name: 2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C2.tif, shape: (82, 512, 512)
Image name: 2015.01.17_MTX_1h-1h_S_V_Pos001_S003_9_C2.tif, sha

## Features extraction

In [ ]:
image_dir = "./BDS3_2025_data/MXT/"
label_dir = "./labels/"
output_dir = "./data/"

os.makedirs(output_dir, exist_ok=True)

In [ ]:
def objects_features(labels, image):
    props = regionprops_table(
        labels,
        intensity_image=image,
        properties=('label', 'area', 'centroid', 'mean_intensity', 'max_intensity', 'min_intensity')
    )
    
    df_features = pd.DataFrame(props)

    # entropy calculation + std
    entropy_list = []
    sphericity_list = []
    std_list = []

    props_full = regionprops(labels, intensity_image=image)

    def entropy(intensities):
        if len(intensities) == 0:
            return 0
        hist_range = (float(intensities.min()), float(intensities.max()))
        if hist_range[0] == hist_range[1]:
            hist_range = (hist_range[0], hist_range[0] + 1e-6)
        hist, _ = np.histogram(intensities, bins=256, range=hist_range, density=True)
        hist = hist[hist > 0]
        return -np.sum(hist * np.log2(hist))

    for region in props_full:
        mask = labels == region.label
        intensities = image[mask]

        # std
        std_list.append(np.std(intensities))

        # entropy
        entropy_list.append(entropy(intensities))

        # sphericity
        if hasattr(region, "surface_area"):
            V = region.area
            A = region.surface_area
            phi = (np.pi**(1/3)) * (6*V)**(2/3) / A if A > 0 else np.nan
        else:
            phi = np.nan
        sphericity_list.append(phi)

    # add into dataframe
    df_features.rename(columns={"area": "volume_vox"}, inplace=True)
    df_features["intensity_std"] = std_list
    df_features["entropy"] = entropy_list
    df_features["sphericity"] = sphericity_list

    # total_intensity
    df_features["total_intensity"] = df_features["mean_intensity"] * df_features["volume_vox"]

    # background correction
    background = np.median(image[labels == 0])
    df_features["background_corrected_intensity"] = (
        df_features["total_intensity"] - background * df_features["volume_vox"]
    )

    # conversion to μm³
    voxel_volume_um3 = 0.06 * 0.06 * 0.12
    df_features["volume_um3"] = df_features["volume_vox"] * voxel_volume_um3

    return df_features


In [ ]:
for fname in os.listdir(image_dir):
    if fname.endswith(".tif"):
        # path to image and label
        image_path = os.path.join(image_dir, fname)
        label_path = os.path.join(label_dir, fname.replace(".tif", "_labels.tif"))

        if not os.path.exists(label_path):
            print(f"[!] No labels for {fname}")
            continue

        print(f"[+] In progress: {fname}")

        # read
        image = imread(image_path)
        labels = imread(label_path)

        # features counting
        df_features = objects_features(labels, image)

        # path to save
        out_csv = os.path.join(output_dir, f"object_features_{fname.replace('.tif','')}.csv")

        # save
        df_features.to_csv(out_csv, index=False)
        print(f"[✓] Saved: {out_csv}")
